In [ ]:
from bs4 import BeautifulSoup as bs
from lxml import html
import requests
import pymongo
from splinter import Browser
import time
import pandas as pd
import numpy as np
from random import random


## initialize browser and login

In [ ]:
# initialize browser
executable_path = {"executable_path": "chromedriver.exe"}
browser = Browser("chrome", **executable_path, headless=False)

### insert profile_id # to search HERE

In [ ]:
#define login url (need to add username and password) 
#define sites to visit after being logged in

pof_login_url = 'https://login.pof.com'

# update with new profile id here
pof_url = 'https://www.pof.com/viewprofile.aspx?profile_id=76643805'

# currently online url
pof_online = 'https://www.pof.com/everyoneonline.aspx'

# base url
pof_base = 'https://www.pof.com/'

# base next_page
pof_nextpage = 'everyoneonline.aspx?page_id='

### input login credentials HERE

In [ ]:
browser.visit(pof_login_url) #visit login page

# add credentials
browser.fill("username", "akela_1111") 
browser.fill("password", "plentyoffish")

browser.click_link_by_id("logincontrol_submitbutton") # click submit to enter credentials

### Create list of profile ids/urls of who is currently online

In [1]:
random(9)

NameError: name 'random' is not defined

In [ ]:
browser.visit(pof_online) # everyone who is online - page 1

In [ ]:
count = 0
profile_urls = []
page = 2


while len(profile_urls) < 75:
    
    html = browser.html
    soup = bs(html, "html.parser")
    
    
    online_results = soup.find_all("div", class_="rc")
    
    
    
    for info in online_results:
        profile_page = pof_base + info.find('a')['href']
        
        if profile_page not in profile_urls:
            
            profile_urls.append(profile_page)
            print(profile_page)
            count += 1
        else:
            continue
    
    page += 1
    new_page = pof_base + pof_nextpage + str(page)
    browser.visit(new_page)



In [ ]:
## visit each profile to scrape

for url in profile_urls:
    browser.visit(url)
    
    html = browser.html
    soup = bs(html, "html.parser")

In [ ]:
profile_dict = {}  ## dictionary to save single profile data collected below

## username and headline

In [ ]:
# use bs and splinter to find username and headline
user = soup.find(id = "username").text
headline = soup.find(id = "headline").text

print(user)
print(headline)

#add to dict
profile_dict["username"] = user
profile_dict["headline"] = headline

## Info in profile table 1


In [ ]:
# use bs and splinter to find first table of profile information
# this is done in parts
# part 1: find header names for first profile table

about_head = soup.find_all("div", class_= "profileheadtitle")
headers = []
for info in about_head:
    headers.append(info.text.replace("\n", ""))
headers

In [ ]:
# part 2: find first half of content (left side of table)
about_content_1 = soup.find_all("div", class_="profileheadcontent")
content = []
for info in about_content_1:
    content.append(info.text.replace("\n\n", "").replace("\n", " "))
len(content)

In [ ]:
# part 3: find 2nd half of content (right side of table)

about_content_2 = soup.find_all("div", class_="profileheadcontent-narrow")
content_2 = []
for info in about_content_2:
    content_2.append(info.text.replace("\n\n", "").replace("\n", " "))
len(content_2)

In [ ]:
# part 4: combine both content into one list
all_content = np.array([content, content_2])
all_content_ordered = np.ravel(all_content, order = "F")
all_content_ordered

In [ ]:
# define dictionary to hold data in the first profile table. keys = header/title, value = content

profile_info_1 = dict(zip(headers, all_content_ordered))
profile_info_1

In [ ]:
# add to dict
profile_dict["profile_info_1"] = profile_info_1

## Info in profile table 2

In [ ]:
about_head_2 = soup.find_all("span", class_="headline")

headers_2 = []
for i in range(9, 25):
    headers_2.append(about_head_2[i].text.replace("\n",""))
    
headers_2

In [ ]:
about_content_3 = soup.find_all("table")[1]("span", class_="txtGrey")
content_3 = []
for info in about_content_3:
    content_3.append(info.text.replace("\n", ""))
content_3

In [ ]:
profile_info_2 = dict(zip(headers_2, content_3))
profile_info_2

In [ ]:
# add to dict
profile_dict["profile_info_2"] = profile_info_2

## Interests

In [ ]:
# interests
try:
    interests = soup.find(id="interests").text.split("\n")[1:4]
except: 
    interests = "None"
    print("No interests were entered")

interests

In [ ]:
# add to dict
profile_dict["interests"] = interests

## About Me

In [ ]:
about_me = soup.find(id="description").text.replace("\n", "")
about_me_split = about_me.split(" ")
about_me_split


In [ ]:
profile_dict["about_me_text"] = about_me
profile_dict["about_me_split"] = about_me_split

In [ ]:
#pd.read_html(pof_url) #403 error - Forbidden???

## dictionary with data of profile_1

In [ ]:
profile_dict